In [28]:
import glob
import re
import os
import math
from nltk.tokenize import word_tokenize
import numpy as np

In [29]:
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex, '', text)
    return text_returned

In [30]:
def finding_all_unique_words_and_freq(dict_global, words):
    for word in set(words):
        if word in dict_global.keys():
            dict_global[word] += words.count(word)
        else:
            dict_global[word] = words.count(word)

In [31]:
file_folder = 'data/*'
dict_global = {}
words_in_doc = {}
docs_mapping = []
N = 0
for file in sorted(glob.glob(file_folder)):
    filename = file
    file = open(file, "r")
    text = file.read()    
    text = remove_special_characters(text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    finding_all_unique_words_and_freq(dict_global, words)
    idx = os.path.basename(filename)    
    docs_mapping.append(idx)
    words_in_doc[N] = words
    print('Filtered tokens:', words)
    N += 1
    file.close()

unique_words_all = sorted(set(dict_global.keys()))
print(unique_words_all)

Filtered tokens: ['to', 'do', 'is', 'to', 'be', 'to', 'be', 'is', 'to', 'do']
Filtered tokens: ['to', 'be', 'or', 'not', 'to', 'be', 'i', 'am', 'what', 'i', 'am']
Filtered tokens: ['i', 'think', 'therefore', 'i', 'am', 'do', 'be', 'do', 'be', 'do']
Filtered tokens: ['do', 'do', 'do', 'da', 'da', 'da', 'let', 'it', 'be', 'let', 'it', 'be']
['am', 'be', 'da', 'do', 'i', 'is', 'it', 'let', 'not', 'or', 'therefore', 'think', 'to', 'what']


In [32]:
class Node:
    def __init__(self, docId, freq):
        self.freq = freq
        self.doc = docId
        self.next = None
    
    def __str__(self):        
        return 'doc:' + str(self.doc) + ', freq:' + str(self.freq)

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.n_docs = 0
    
    def print_list(self):
        aux = self.head
        while aux:
            print(aux)
            aux = aux.next
    
    def get_doclist(self):
        l = []
        aux = self.head
        while aux:
            l.append([aux.doc, aux.freq])
            aux = aux.next
        return l
    
    def add_doc(self, doc, freq):
        node = Node(doc, freq)        
        if self.head == None:
            self.head = node        
        else:
            self.tail.next = node
        self.tail = node
        self.n_docs += 1


In [33]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = LinkedList()

for docID in range(len(docs_mapping)):
    words = words_in_doc[docID]
    for word in set(words):
        linked_list_data[word].add_doc(docID, words.count(word))        

linked_list_data['do'].print_list()

doc:0, freq:2
doc:2, freq:3
doc:3, freq:3


In [37]:
print(docs_mapping)
m = np.zeros((len(unique_words_all), len(docs_mapping)))
for i in range(len(unique_words_all)):
    word = unique_words_all[i]
    postings = linked_list_data[word].get_doclist()
    ni = linked_list_data[word].n_docs
    for node in postings:
        docID = node[0]
        freq = node[1]
        m[i, docID] = freq
    print(word, m[i])

['doc1.txt', 'doc2.txt', 'doc3.txt', 'doc4.txt']
am [0. 2. 1. 0.]
be [2. 2. 2. 2.]
da [0. 0. 0. 3.]
do [2. 0. 3. 3.]
i [0. 2. 2. 0.]
is [2. 0. 0. 0.]
it [0. 0. 0. 2.]
let [0. 0. 0. 2.]
not [0. 1. 0. 0.]
or [0. 1. 0. 0.]
therefore [0. 0. 1. 0.]
think [0. 0. 1. 0.]
to [4. 2. 0. 0.]
what [0. 1. 0. 0.]


In [40]:
print(docs_mapping)
m = np.zeros((len(unique_words_all), len(docs_mapping)))
for i in range(len(unique_words_all)):
    word = unique_words_all[i]
    postings = linked_list_data[word].get_doclist()
    ni = linked_list_data[word].n_docs
    idf = math.log2(N/ni)
    for node in postings:
        docID = node[0]
        freq = node[1]
        m[i, docID] = (1 + math.log2(freq))*idf
    print(word, m[i])

['doc1.txt', 'doc2.txt', 'doc3.txt', 'doc4.txt']
am [0. 2. 1. 0.]
be [0. 0. 0. 0.]
da [0.       0.       0.       5.169925]
do [0.830075   0.         1.07285637 1.07285637]
i [0. 2. 2. 0.]
is [4. 0. 0. 0.]
it [0. 0. 0. 4.]
let [0. 0. 0. 4.]
not [0. 2. 0. 0.]
or [0. 2. 0. 0.]
therefore [0. 0. 2. 0.]
think [0. 0. 2. 0.]
to [3. 2. 0. 0.]
what [0. 2. 0. 0.]


In [52]:
norm = np.sum(m**2, axis=0)
norm = [math.sqrt(norm[i]) for i in range(len(norm))]
print(norm)

[5.068434127344514, 4.898979485566356, 3.761784256839167, 7.738161623767062]


In [56]:
query = 'to do'
query = remove_special_characters(query)
query = word_tokenize(query)
query = [q.lower() for q in query]
q_vector = np.zeros(len(unique_words_all))
for i in range(len(unique_words_all)):
    word = unique_words_all[i]
    if word in query:
        ni = linked_list_data[word].n_docs
        idf = math.log2(N/ni)
        q_vector[i] = (1 + math.log2(query.count(word)))*idf
print(q_vector)

[0.        0.        0.        0.4150375 0.        0.        0.
 0.        0.        0.        0.        0.        1.        0.       ]


In [69]:
ranking = np.zeros(N)
for j in range(N):
    ranking[j] = np.dot(m[:,j], q_vector) / norm[j]
    print(docs_mapping[j], ranking[j])

doc1.txt 0.6598709123142042
doc2.txt 0.4082482904638631
doc3.txt 0.11836819852778786
doc4.txt 0.05754281796914421
